In [ ]:
# random proxy taken from https://codelike.pro/create-a-crawler-with-rotating-ip-proxy-in-python/

In [347]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import random
import time

In [4]:
urls = ['https://www.basketball-reference.com/boxscores/?month=01&day=18&year=2019',
        'https://www.basketball-reference.com/boxscores/?month=01&day=19&year=2019']

In [5]:
k = 3

In [14]:
site_url = 'https://www.basketball-reference.com'

In [15]:
def fluff_number(x, digits = 2):
    y = str(x)
    gap = digits - len(str(x))
    if gap > 0:
        for i in range(gap):
            y = '0' + y
    return(y)

In [331]:
def random_proxy(proxies):
    return random.randint(0, len(proxies) - 1)

def open_with_proxy(u, max_tries = 20):
    ua = UserAgent() # From here we generate a random user agent
    proxies = [] # Will contain proxies [ip, port]

    # Retrieve latest proxies
    proxies_req = Request('https://www.sslproxies.org/')
    proxies_req.add_header('User-Agent', ua.random)
    proxies_doc = urlopen(proxies_req).read().decode('utf8')

    soup = BeautifulSoup(proxies_doc, 'html.parser')
    proxies_table = soup.find(id='proxylisttable')

  # Save proxies in the array
    for row in proxies_table.tbody.find_all('tr'):
        proxies.append({
            'ip':   row.find_all('td')[0].string,
            'port': row.find_all('td')[1].string 
      })

    connected = False
    num_tries = 0

    while connected is False:
        # Generate a random proxy
        proxy_index = random_proxy(proxies)
        proxy = proxies[proxy_index]
        num_tries += 1
    
        req = Request(u)
        req.set_proxy(proxy['ip'] + ':' + proxy['port'], 'http')

        try:
            html = urlopen(req).read().decode('utf8')
            connected = True
            print('Connected with ' + str(num_tries) + ' tries')
            return(html)
        except: # If error, delete this proxy and find another one
            del proxies[proxy_index]
            if num_tries > 20:
                print('Unable to connect. Max tries reached')
                return None

In [340]:
def get_games_from_date(month, day, year):
    site_url = 'http://www.basketball-reference.com'
    u = 'http://www.basketball-reference.com/boxscores/?'
    u += 'month=' + fluff_number(month)
    u += '&day=' + fluff_number(day)
    u += '&year=' + str(year)
    print(u)
    #req = Request(u)
    #req.set_proxy(proxy['ip'] + ':' + proxy['port'], 'http')
    #html = urlopen(req)
    html = open_with_proxy(u)
    soup = BeautifulSoup(html)
    links = [site_url + x['href'] for x in soup.find_all('a', text = 'Box Score')]
    return(links)

In [341]:
def get_stat_from_tr(tr):
    return([tr.find('th')['csk'], dict([(x['data-stat'], x.text) for x in tr.find_all('td')])])

In [348]:
def get_table_from_game(u):
    game_name = u.split('/')[-1].split('.')[0]
    #proxy_index = random_proxy()
    #proxy = proxies[proxy_index]

    #req = Request(u)
    #req.set_proxy(proxy['ip'] + ':' + proxy['port'], 'http')

    #html = urlopen(req)
    time.sleep(5)
    html = open_with_proxy(u)
    soup = BeautifulSoup(html)
    tables = soup.find_all('table', attrs={'class': 'stats_table'})
    table_tuples = [(tbl['id'].split('_')[1], [tr for tr in tbl.find_all('tr') if tr.find('th').has_attr('csk')]) for tbl in tables]
    # Get all of the rows whose first header contains the csk attribute
    # trs = [tr for tr in soup.find_all('tr') if tr.find('th').has_attr('csk')]
    data = [(tbl_tup[0], [get_stat_from_tr(tr) for tr in tbl_tup[1]]) for tbl_tup in table_tuples]

    d = {}
    for tbl in data:
        team = tbl[0]
        for player in tbl[1]:
            player_name = player[0]
            d[player_name] = {**{'team': team, 'game': game_name}, **player[1]}
    df = pd.DataFrame(d).T
    return(df)

In [359]:
def get_data_from_date(month, day, year):
    data = pd.concat([get_table_from_game(game) for game in get_games_from_date(month, day, year)])
    data['year'] = year
    data['month'] = month
    data['day'] = day
    data['date'] = str(year) + '-' + fluff_number(month) + '-' + fluff_number(day)
    data['player'] = data.index
    data = data.set_index(['game', 'player'])
    return(data)

In [360]:
games_data = get_data_from_date(1,19,2019)

http://www.basketball-reference.com/boxscores/?month=01&day=19&year=2019
Connected with 1 tries
Connected with 2 tries
Connected with 1 tries
Connected with 1 tries
Connected with 2 tries
Connected with 2 tries
Connected with 1 tries
Connected with 1 tries
Connected with 1 tries
Connected with 1 tries
Connected with 1 tries


/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [363]:
import sqlalchemy
database_username = 'root'
database_password = 'welcome2sql'
database_ip       = '192.168.0.229:6603'
database_name     = 'nba'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [ ]:
games_data.to_sql(con=database_connection, name='games_players', if_exists='replace')

In [361]:
games_data.to_sql()

ast_pct blk_pct def_rtg drb_pct efg_pct  \
game         player                                                          
201901190ATL Irving,Kyrie             26.8     0.0     111     5.8    .711   
             Tatum,Jayson              9.5     8.2     104    12.3    .538   
             Morris,Marcus             5.0     3.3     102    22.6    .400   
             Horford,Al                0.0     6.8     105    19.4    .409   
             Smart,Marcus             30.7     0.0     109     0.0    .250   
             Rozier,Terry             20.3     3.2      90    36.5    .375   
             Brown,Jaylen             10.2     0.0      98    15.3    .400   
             Baynes,Aron               0.0     5.9     110     6.7    .750   
             Ojeleye,Semi              0.0     0.0     116     0.0   1.000   
             Theis,Daniel              0.0     0.0     116     0.0   1.250   
             Wanamaker,Brad           22.0     0.0     116     0.0    .000   
             Yabusele,Guerschon        NaN     NaN     NaN     NaN     NaN   
             Williams,Robert           NaN     NaN     NaN     NaN     NaN   
             Huerter,Kevin            31.8     0.0     110     5.9    .500   
             Young,Trae               31.4     0.0     118     3.0    .412   
             Bembry,DeAndre'           4.6     0.0     117    16.5    .500   
             Collins,John             15.9     0.0     113    28.6    .786   
             Dedmon,Dewayne            0.0     3.5     116    12.4    .417   
             Waller-Prince,Taurean     6.0     3.7     102    26.1    .250   
             Lin,Jeremy               38.9     4.1     112     4.9    .286   
             Len,Alex                 22.8     4.5     106    21.6    .750   
             Spellman,Omari            0.0     5.6     108    33.2   1.000   
             Carter,Vince             19.9    10.2     111    12.1   1.000   
             Hamilton,Daniel           NaN     NaN     NaN     NaN     NaN   
             Anderson,Justin           NaN     NaN     NaN     NaN     NaN   
             Dorsey,Tyler              NaN     NaN     NaN     NaN     NaN   
201901190CHI Richardson,Josh          11.2     0.0     111     9.5    .800   
             Winslow,Justise          20.3     0.0     103    18.6    .571   
             Johnson,James             9.7     0.0     106    11.2    .583   
             Whiteside,Hassan         14.7     9.3      99    22.6    .750   
...                                    ...     ...     ...     ...     ...   
201901190PHI Shamet,Landry            11.8     0.0     124     7.2   1.500   
             Bolden,Jonah             10.9     6.0     111    32.6    .750   
             Johnson,Amir              NaN     NaN     NaN     NaN     NaN   
             Brewer,Corey              NaN     NaN     NaN     NaN     NaN   
             Korkmaz,Furkan            NaN     NaN     NaN     NaN     NaN   
201901190TOR Holiday,Justin           16.5     0.0     120    15.6    .438   
             Jackson,Jaren             7.0     0.0     126    24.6    .542   
             Temple,Garrett            6.6     3.4     122     9.1    .667   
             Conley,Mike              27.6     0.0     124    10.4    .556   
             Green,JaMychal           11.6     0.0     123    34.5    .250   
             Casspi,Omri               5.6     3.1     121    16.6    .200   
             Mack,Shelvin             36.8     0.0     126     4.5    .750   
             Carter,Jevon             24.2     0.0     126     4.5    .429   
             Rabb,Ivan                 9.6     4.0     120    37.6    .500   
             Watanabe,Yuta             9.7     0.0     130     6.7    .500   
             Gasol,Marc                NaN     NaN     NaN     NaN     NaN   
             Noah,Joakim               NaN     NaN     NaN     NaN     NaN   
             Lowry,Kyle               29.6     3.2      96    12.4    .438   
             Siakam,Pascal            10.4     0.0      9

In [2]:
# Main function
def scrape(u):
    
    ua = UserAgent() # From here we generate a random user agent
    proxies = [] # Will contain proxies [ip, port]

    # Retrieve latest proxies
    proxies_req = Request('https://www.sslproxies.org/')
    proxies_req.add_header('User-Agent', ua.random)
    proxies_doc = urlopen(proxies_req).read().decode('utf8')

    soup = BeautifulSoup(proxies_doc, 'html.parser')
    proxies_table = soup.find(id='proxylisttable')
a
  # Save proxies in the array
    for row in proxies_table.tbody.find_all('tr'):
        proxies.append({
            'ip':   row.find_all('td')[0].string,
            'port': row.find_all('td')[1].string 
      })

  # Choose a random proxy
    proxy_index = random_proxy()
    proxy = proxies[proxy_index]

    for n in range(1, 100):
        req = Request(u)
        req.set_proxy(proxy['ip'] + ':' + proxy['port'], 'http')

        # Every k requests, generate a new proxy
        if n % k == 0:
            proxy_index = random_proxy()
            proxy = proxies[proxy_index]

        # Make the call
        try:
            my_ip = urlopen(req).read().decode('utf8')
            print('#' + str(n) + ': ' + my_ip)
        except: # If error, delete this proxy and find another one
            del proxies[proxy_index]
            print('Proxy ' + proxy['ip'] + ':' + proxy['port'] + ' deleted.')
            proxy_index = random_proxy()
            proxy = proxies[proxy_index]

# Retrieve a random index proxy (we need the index to delete it if not working)


In [3]:
if __name__ == '__main__':
  main()

Proxy 41.33.31.38:40581 deleted.
#2: 157.230.83.203

#3: 157.230.83.203

#4: 157.230.83.203

#5: 157.230.83.203

#6: 157.230.83.203

#7: 157.230.83.203

#8: 157.230.83.203

#9: 157.230.83.203

#10: 157.230.83.203

#11: 77.120.163.23

#12: 77.120.163.23

#13: 77.120.163.23

#14: 77.120.163.23

#15: 77.120.163.23

#16: 77.120.163.23

#17: 77.120.163.23

#18: 77.120.163.23

#19: 77.120.163.23

#20: 77.120.163.23

#21: 77.120.163.87

#22: 77.120.163.87

#23: 77.120.163.87

#24: 77.120.163.87

#25: 77.120.163.87

#26: 77.120.163.87

#27: 77.120.163.87

#28: 77.120.163.87

#29: 77.120.163.87

#30: 77.120.163.87

#31: 178.128.210.1

#32: 178.128.210.1

#33: 178.128.210.1

#34: 178.128.210.1

#35: 178.128.210.1

#36: 178.128.210.1

#37: 178.128.210.1

#38: 178.128.210.1

#39: 178.128.210.1

#40: 178.128.210.1

#41: 77.120.163.23

#42: 77.120.163.23

#43: 77.120.163.23

#44: 77.120.163.23

#45: 77.120.163.23

#46: 77.120.163.23

#47: 77.120.163.23

#48: 77.120.163.23

#49: 77.120.163.23

#50: 7